In [3]:
!pip install tensorflow
!pip install scikit-image

In [49]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, ResNet50
from sklearn.preprocessing import normalize
import pickle
import requests
from PIL import Image
from io import BytesIO
import ast
from keras.preprocessing import image as keras_image

In [50]:
data = pd.read_csv('A2_Data.csv')

## Image Evaluation

Feature Extraction

In [51]:
# Step 2: Preprocess the images
def preprocess_image(img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

# Step 3: Utilize a pre-trained Convolutional Neural Network (CNN) to extract features
def extract_features(img):
    model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
    img = preprocess_input(np.array(img.resize((224, 224))))
    features = model.predict(np.expand_dims(img, axis=0))
    return features

# Step 4: Normalize the extracted features
def normalize_features(features):
    features = features.reshape(1, -1)
    normalized_features = normalize(features)
    return normalized_features

## TO BE RUN ONLY ONCE - FOR FEATURE EXTRACTION

# # Process all images and extract features
# image_features = []
# for img_link_str in data['Image']:
#     # Convert the string representation of the list to an actual list
#     img_link_list = ast.literal_eval(img_link_str)

#     for img_link in img_link_list:
#         if isinstance(img_link, str) and img_link.startswith('http'):
#             response = requests.get(img_link)
#             if response.status_code == 200:
#                 img = Image.open(BytesIO(response.content))
#                 img_features = extract_features(img)
#                 normalized_features = normalize_features(img_features)
#                 image_features.append(normalized_features)
#             else:
#                 print("Failed to fetch image:", img_link)
#         else:
#             print("Invalid image link:", img_link)

# image_features = np.array(image_features)

In [52]:
with open('image_features.pkl', 'rb') as f:
    image_features = pickle.load(f)

In [53]:
image_features

array([[[0.00000000e+00, 9.46111530e-02, 0.00000000e+00, ...,
         2.75597791e-03, 1.81075018e-02, 9.88949761e-02]],

       [[1.33375940e-03, 1.21552069e-02, 0.00000000e+00, ...,
         1.16400690e-04, 6.37179334e-03, 5.59440367e-02]],

       [[3.37673337e-05, 4.36395332e-02, 3.26262350e-04, ...,
         2.26968224e-03, 1.33218905e-02, 7.04480857e-02]],

       ...,

       [[0.00000000e+00, 1.34535888e-02, 7.98366847e-04, ...,
         2.92027108e-02, 8.39617010e-03, 4.30044569e-02]],

       [[6.29557937e-04, 0.00000000e+00, 0.00000000e+00, ...,
         3.37668834e-03, 1.68580725e-03, 7.47690769e-03]],

       [[0.00000000e+00, 5.50508462e-02, 4.37362978e-05, ...,
         2.02349667e-03, 9.75323096e-03, 3.08927931e-02]]], dtype=float32)

In [54]:
len(image_features)

1640

Dictionary to store the URL's

In [55]:
import requests
from PIL import Image
from io import BytesIO
import ast

# Dictionary to store lists of image URLs for each index
image_index_to_urls = {}

# Process all images and store image URLs with indices
for index, img_link_str in enumerate(data['Image']):
    # Convert the string representation of the list to an actual list
    img_link_list = ast.literal_eval(img_link_str)
    image_urls = []
    
    for img_link in img_link_list:
        # Check if the link is valid and starts with 'http'
        if isinstance(img_link, str) and img_link.startswith('http'):
            # Fetch the image
            response = requests.get(img_link)
            if response.status_code == 200:
                # Open the image and store the URL
                img = Image.open(BytesIO(response.content))
                image_urls.append(img_link)
            else:
                print("Failed to fetch image:", img_link)
        else:
            print("Invalid image link:", img_link)
    
    # Store the list of image URLs for the current index
    image_index_to_urls[index] = image_urls

# Check the total number of images fetched
total_images = sum(len(urls) for urls in image_index_to_urls.values())
print("Total number of images fetched:", total_images)

Failed to fetch image: https://images-na.ssl-images-amazon.com/images/I/71F3npeHUDL._SY88.jpg
Failed to fetch image: https://images-na.ssl-images-amazon.com/images/I/71wHUWncMGL._SY88.jpg
Failed to fetch image: https://images-na.ssl-images-amazon.com/images/I/71B8OOE5N8L._SY88.jpg
Failed to fetch image: https://images-na.ssl-images-amazon.com/images/I/81SX3oAWbNL._SY88.jpg
Failed to fetch image: https://images-na.ssl-images-amazon.com/images/I/718niQ1GEwL._SY88.jpg
Failed to fetch image: https://images-na.ssl-images-amazon.com/images/I/61OboZT-kcL._SY88.jpg
Failed to fetch image: https://images-na.ssl-images-amazon.com/images/I/710a2Pyh5lL._SY88.jpg
Failed to fetch image: https://images-na.ssl-images-amazon.com/images/I/816NMd0LexL._SY88.jpg
Total number of images fetched: 1640


In [56]:
image_index_to_urls

{0: ['https://images-na.ssl-images-amazon.com/images/I/81q5+IxFVUL._SY88.jpg'],
 1: ['https://images-na.ssl-images-amazon.com/images/I/71HSx4Y-5dL._SY88.jpg',
  'https://images-na.ssl-images-amazon.com/images/I/71dVsYejzTL._SY88.jpg',
  'https://images-na.ssl-images-amazon.com/images/I/71domStNfIL._SY88.jpg'],
 2: ['https://images-na.ssl-images-amazon.com/images/I/71Md5ihUFLL._SY88.jpg'],
 3: ['https://images-na.ssl-images-amazon.com/images/I/71Isri9SEaL._SY88.jpg'],
 4: ['https://images-na.ssl-images-amazon.com/images/I/71w8aOdrTuL._SY88.jpg'],
 5: ['https://images-na.ssl-images-amazon.com/images/I/81dxkALs4CL._SY88.jpg',
  'https://images-na.ssl-images-amazon.com/images/I/81VixyruzDL._SY88.jpg',
  'https://images-na.ssl-images-amazon.com/images/I/51IWGFH3IPL._SY88.jpg',
  'https://images-na.ssl-images-amazon.com/images/I/81p1k-2194L._SY88.jpg'],
 6: ['https://images-na.ssl-images-amazon.com/images/I/71cS64LddWL._SY88.jpg'],
 7: ['https://images-na.ssl-images-amazon.com/images/I/71z9b

In [57]:
# Specify the file path where you want to save the image features
pickle_file_path = 'image_index_to_urls.pkl'
# Serialize and save the image_features array to a file
with open(pickle_file_path, 'wb') as f:
    pickle.dump(image_index_to_urls, f)

In [58]:
with open('image_index_to_urls.pkl', 'rb') as f:
    image_index_to_urls = pickle.load(f)

Extracting features of Single Image (Input)

In [59]:
import requests
from PIL import Image
from io import BytesIO

def process_single_image(img_link):
    if isinstance(img_link, str) and img_link.startswith('http'):
        response = requests.get(img_link)
        if response.status_code == 200:
            img = Image.open(BytesIO(response.content))
            img_features = extract_features(img)
            normalized_features = normalize_features(img_features)
            return normalized_features
        else:
            print("Failed to fetch image:", img_link)
    else:
        print("Invalid image link:", img_link)
        return None

# Example usage
img_link = "https://images-na.ssl-images-amazon.com/images/I/81q5+IxFVUL._SY88.jpg"
image_featuresSingle = process_single_image(img_link)


1/1 [==============================] - 1s 1s/step


In [60]:
image_featuresSingle

array([[0.        , 0.09461115, 0.        , ..., 0.00275598, 0.0181075 ,
        0.09889498]], dtype=float32)

Dictionary for Review storage

In [61]:
# Assuming you have a 'Review' column in your data DataFrame
image_index_to_reviews = {}

# Process all reviews and store them with indices
for index, review_str in enumerate(data['Review Text']):
    image_index_to_reviews[index] = review_str


In [62]:
image_index_to_reviews

{0: 'Loving these vintage springs on my vintage strat. They have a good tension and great stability. If you are floating your bridge and want the most out of your springs than these are the way to go.',
 1: "Works great as a guitar bench mat. Not rugged enough for abuse but if you take care of it, it will take care of you. Makes organization of workspace much easier because screws won't roll around. Color is good too.",
 2: "We use these for everything from our acoustic bass down to our ukuleles. I know there is a smaller model available for ukes, violins, etc.; we haven't yet ordered those, but these will work on smaller instruments if one doesn't extend the feet to their maximum width. They're gentle on the instruments, and the grippy material keeps them secure.\n\nThe greatest benefit has been when writing music at the computer and needing to set a guitar down to use the keyboard/mouse - just easier for me than a hanging stand.\n\nWe have several and gave one to a friend for Christm

Dictionary Creation for mapping of URL's and Review

In [63]:
import pandas as pd
import ast

# Dictionary to store image URLs and reviews with new indices
image_data = {}
new_index = 0

# Iterate through each row in the DataFrame
for index, row in data.iterrows():
    # Extract image URLs and review from the row
    img_links = ast.literal_eval(row['Image'])
    review = row['Review Text']
    
    # Store the review for each image URL
    for img_link in img_links:
        if isinstance(img_link, str) and img_link.startswith('http'):
            # Add the image URL and review to the dictionary with new index
            image_data[new_index] = [(img_link, review)]
            new_index += 1
        else:
            print("Invalid image link:", img_link)

# Now image_data contains a dictionary where each key is a new index
# and the corresponding value is a list of tuples, each containing
# an image URL and its associated review.

# # Example usage:
# for index, images_and_review in image_data.items():
#     print("Index:", index)
#     for img_url, review in images_and_review:
#         print("Image URL:", img_url)
#         print("Review:", review)
#     print()


In [64]:
keys = [128, 129, 208, 209, 900, 1168, 1421, 1544]

for key in keys:
    image_data.pop(key, None)

In [65]:
len(image_data)

1640

### Cosine Similarity | Composite Similarity | Image-based Retrieval 

In [81]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

def find_similar_images(input_features, input_review, image_data, image_features, top_n=3):
    similarity_scores = {}
    
    # Iterate over each index and its corresponding images and reviews
    for index, images_and_review in image_data.items():
        # Check if the index is within bounds
        if index < len(image_features):
            for img_url, review in images_and_review:
                # Calculate cosine similarity between input image features and the features of images at this index
                similarity_score = cosine_similarity(input_features, image_features[index].reshape(1, -1))[0][0]
                similarity_scores[img_url] = (similarity_score, review)
    
    # Sort images based on cosine similarity scores
    sorted_images = sorted(similarity_scores.items(), key=lambda x: x[1][0], reverse=True)
    
    # Select top N similar images
    top_similar_images = sorted_images[1:4]
    
    return top_similar_images

input_review = input("Enter your review: ")
input_image_url = input("Enter your image URL: ")

# Preprocess the input review
preprocessed_input_review = preprocess_text(input_review)
# Replace other types of missing values with numpy NaN values
data['Review Text'].replace(['None', 'NaN'], np.nan, inplace=True)

# Fill NaN values with empty strings
data['Review Text'] = data['Review Text'].fillna('')

# Apply pre-processing functions to the text data
data['Processed_Text'] = data['Review Text'].apply(preprocess_text)

# Initialize and fit the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(data['Processed_Text'])
input_review_vector = tfidf_vectorizer.transform([preprocessed_input_review])

# Find top three similar images
top_similar_images = find_similar_images(input_image_features, input_review, image_data, image_features, top_n=3)

composite_similarity_data = []
# Print the URLs and reviews of top similar images
for i, (img_url, (similarity_score, review)) in enumerate(top_similar_images, start=1):
    print(f"Top similar image {i}:")
    print("Image URL:", img_url)
    print("Similarity Score:", similarity_score)
    print("Review:", review)
    
    # Calculate cosine similarity between input review and current review
    preprocessed_review = preprocess_text(review)  # Preprocess the review
    input_review_vector = tfidf_vectorizer.transform([preprocessed_input_review])  # Convert input review to TF-IDF vector
    review_vector = tfidf_vectorizer.transform([preprocessed_review])  # Convert review to TF-IDF vector
    cosine_sim = cosine_similarity(input_review_vector, review_vector)[0][0]
    print("Cosine similarity of text:", cosine_sim)
    composite_cosine_image = (similarity_score+cosine_sim)/2
    #composite_similarity_scores_Image.append((img_url, similarity_score, cosine_sim, composite_cosine_image))
    print("Composite similarity:", composite_cosine_image)
    print()
    image_dict = {}
    image_dict["image_url"] = img_url
    image_dict["review_text"] = review
    image_dict["cosine_image"] = similarity_score
    image_dict["cosine_text"] = cosine_sim
    image_dict["composite"] = composite_cosine_image
    composite_similarity_data.append(image_dict)

Enter your review: Loving these vintage springs on my vintage strat. They have a good tension and great stability. If you are floating your bridge and want the most out of your springs than these are the way to go.
Enter your image URL: https://images-na.ssl-images-amazon.com/images/I/81q5+IxFVUL._SY88.jpg
Top similar image 1:
Image URL: https://images-na.ssl-images-amazon.com/images/I/61Rv7lR1aTL._SY88.jpg
Similarity Score: 0.7400955
Review: A great product, which made a wonderful under-bed light in our application.

- Length is perfect for a kings size frame,  Start about 1 food from the headboard, and this will cover the permitter all the way around, with adjustment for your particular bed

- The dimming feature is great and a must if you will use it for a nightlight application.  In fact, the lights are so bright you have to keep them at or near the dimmest level for an under-bed nightlight application.

- Every LED lit up.  No dead diodes.

Why the deduction?

- The adhesive back 

#  Text Evaluation

In [66]:
!pip install nltk
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/vani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/vani/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/vani/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Preprocessing | TF-IDF Scores

In [67]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Define functions for text preprocessing
def preprocess_text(text):
    # Check if the input is a string
    if isinstance(text, str):
        # Lowercasing
        text = text.lower()
        # Removing punctuations
        text = re.sub(r'[^\w\s]', '', text)
    return text

def tokenize(text):
    # Check if the input is a string
    if isinstance(text, str):
        # Tokenization
        tokens = word_tokenize(text)
    else:
        tokens = []
    return tokens

def remove_stopwords(tokens):
    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]
    return filtered_tokens

def stem(tokens):
    # Stemming
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return stemmed_tokens

def lemmatize(tokens):
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return lemmatized_tokens

# Apply pre-processing functions to the text data
data['Processed_Text'] = data['Review Text'].apply(preprocess_text)
data['Tokens'] = data['Processed_Text'].apply(tokenize)
data['Tokens'] = data['Tokens'].apply(remove_stopwords)
data['Stemmed_Tokens'] = data['Tokens'].apply(stem)
data['Lemmatized_Tokens'] = data['Tokens'].apply(lemmatize)

# Calculate TF for each term in each document
def calculate_tf(tokens):
    term_freq = {}
    for token in tokens:
        term_freq[token] = term_freq.get(token, 0) + 1
    total_tokens = len(tokens)
    tf = {term: freq / total_tokens for term, freq in term_freq.items()}
    return tf

data['TF'] = data['Lemmatized_Tokens'].apply(calculate_tf)

# Calculate IDF for each term in the entire corpus
def calculate_idf(documents):
    doc_freq = {}
    total_documents = len(documents)
    for doc in documents:
        unique_terms = set(doc)
        for term in unique_terms:
            doc_freq[term] = doc_freq.get(term, 0) + 1
    idf = {term: np.log(total_documents / (freq + 1)) for term, freq in doc_freq.items()}
    return idf

idf = calculate_idf(data['Lemmatized_Tokens'])

# Calculate TF-IDF scores for each term in each document
def calculate_tfidf(tf, idf):
    tfidf = {}
    for term, tf_score in tf.items():
        tfidf[term] = tf_score * idf.get(term, 0)
    return tfidf

# Store TF-IDF scores in a dictionary with document ID as the key
tfidf_dict = {}
for idx, row in data.iterrows():
    tfidf_dict[idx] = calculate_tfidf(row['TF'], idf)

# Print TF-IDF scores
for doc_id, tfidf_scores in tfidf_dict.items():
    print(f"Document {doc_id + 1} TF-IDF Scores:")
    print(tfidf_scores)


Document 1 TF-IDF Scores:
{'loving': 0.35322115776986907, 'vintage': 0.5284421733087595, 'spring': 0.6437751649736402, 'strat': 0.22161575603506845, 'good': 0.10564968665624859, 'tension': 0.31403538010972787, 'great': 0.07575427705680808, 'stability': 0.34106638731693883, 'floating': 0.36809739452414975, 'bridge': 0.2234938144995149, 'want': 0.17357934571853118, 'way': 0.17357934571853118, 'go': 0.15622713916762004}
Document 2 TF-IDF Scores:
{'work': 0.07383778809307462, 'great': 0.04940496329791831, 'guitar': 0.05472961047047526, 'bench': 0.25257143436147944, 'mat': 0.25257143436147944, 'rugged': 0.21573239695334015, 'enough': 0.11036119245717894, 'abuse': 0.24006351816792373, 'take': 0.2587764909789401, 'care': 0.36519174590260234, 'make': 0.0936593516467727, 'organization': 0.2702003521053127, 'workspace': 0.2702003521053127, 'much': 0.09403578915734291, 'easier': 0.167966645272118, 'screw': 0.11817828421546572, 'wont': 0.14575683554316188, 'roll': 0.20992668423053484, 'around': 0.

In [68]:
# Specify the file path where you want to save the image features
pickle_file_path = 'tfidf_dict.pkl'

# Serialize and save the image_features array to a file
with open(pickle_file_path, 'wb') as f:
    pickle.dump(tfidf_dict, f)

In [69]:
with open('tfidf_dict.pkl', 'rb') as f:
    tfidf_dict12 = pickle.load(f)

In [70]:
tfidf_dict12

{0: {'loving': 0.35322115776986907,
  'vintage': 0.5284421733087595,
  'spring': 0.6437751649736402,
  'strat': 0.22161575603506845,
  'good': 0.10564968665624859,
  'tension': 0.31403538010972787,
  'great': 0.07575427705680808,
  'stability': 0.34106638731693883,
  'floating': 0.36809739452414975,
  'bridge': 0.2234938144995149,
  'want': 0.17357934571853118,
  'way': 0.17357934571853118,
  'go': 0.15622713916762004},
 1: {'work': 0.07383778809307462,
  'great': 0.04940496329791831,
  'guitar': 0.05472961047047526,
  'bench': 0.25257143436147944,
  'mat': 0.25257143436147944,
  'rugged': 0.21573239695334015,
  'enough': 0.11036119245717894,
  'abuse': 0.24006351816792373,
  'take': 0.2587764909789401,
  'care': 0.36519174590260234,
  'make': 0.0936593516467727,
  'organization': 0.2702003521053127,
  'workspace': 0.2702003521053127,
  'much': 0.09403578915734291,
  'easier': 0.167966645272118,
  'screw': 0.11817828421546572,
  'wont': 0.14575683554316188,
  'roll': 0.2099266842305348

### Cosine Similarity | Composite Similarity | Text-based Retrieval

In [72]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Preprocess the text data and handle missing values
def preprocess_text(text):
    if isinstance(text, str):
        return text.lower()  # Lowercasing the text
    else:
        return ""  # Replace missing values with an empty string

input_review = input("Enter your review: ")
input_image_url = input("Enter your image URL: ")

# Preprocess the input review
preprocessed_input_review = preprocess_text(input_review)

# Combine the input review with other reviews for TF-IDF calculation
all_reviews = [preprocessed_input_review] + [preprocess_text(review) for review in data['Review Text'].tolist()]


# Calculate IDF for all reviews
idf = calculate_idf([preprocess_text(review) for review in data['Review Text'].tolist()])

# Calculate TF-IDF scores for all reviews
tfidf_scores = [calculate_tfidf(calculate_tf(preprocess_text(review)), idf) for review in all_reviews]

# Convert TF-IDF scores to vectors
tfidf_vectors = [np.array([tfidf.get(term, 0) for term in idf.keys()]) for tfidf in tfidf_scores]

# Convert input TF-IDF scores to vector
input_tfidf_vector = tfidf_vectors[0]

# Calculate Cosine Similarity between the input review and all other reviews
cosine_similarities = [cosine_similarity(input_tfidf_vector.reshape(1, -1), tfidf_vector.reshape(1, -1))[0][0] for tfidf_vector in tfidf_vectors[1:]]

# Sort the reviews based on their similarity scores
sorted_indices = np.argsort(cosine_similarities)[::-1]  # Sort in descending order
top_similar_reviews_indices = sorted_indices[1:4]  # Select top 3 most similar reviews



# Get the top three most similar reviews and their similarity scores
top_similar_reviews = []
top_similar_image_urls = []
cosine_similarities_images = []
for index in top_similar_reviews_indices:
    review = all_reviews[index + 1]
    similarity_score = cosine_similarities[index]
    top_similar_reviews.append((review, similarity_score))
    top_similar_image_urls.append(data['Image'][index]) 
    temp_url = eval(data['Image'][index])[0]
    
    # Calculate Cosine Similarity between input image and top similar images
    input_image_features = process_single_image(input_image_url)
    temp_cosine = cosine_similarity(input_image_features.reshape(1, -1), process_single_image(temp_url).reshape(1, -1))[0][0]
    cosine_similarities_images.append(temp_cosine)

# Display and store the top three most similar reviews and their corresponding image URLs
for i, ((review, similarity_score_text), img_url, similarity_score_image) in enumerate(zip(top_similar_reviews, top_similar_image_urls, cosine_similarities_images), start=1):
    print(f"Top similar review {i}:")
    print("Review:", review)
    print("Cosine similarity of text:", similarity_score_text)
    print("Image URL:", img_url)  
    print("Cosine similarity of image:", similarity_score_image)
    composite_cosine = (similarity_score_text+similarity_score_image)/2
   # composite_similarity_scores_Text.append((img_url, similarity_score_text, cosine_sim, composite_cosine))
    print("Composite similarity:", composite_cosine)
    print()
    text_dict = {}
    text_dict["image_url"] = img_url
    text_dict["review_text"] = review
    text_dict["cosine_image"] = similarity_score_image
    text_dict["cosine_text"] = similarity_score_text
    text_dict["composite"] = composite_cosine
    composite_similarity_data.append(text_dict)

Enter your review: Loving these vintage springs on my vintage strat. They have a good tension and great stability. If you are floating your bridge and want the most out of your springs than these are the way to go.
Enter your image URL: https://images-na.ssl-images-amazon.com/images/I/81q5+IxFVUL._SY88.jpg
1/1 [==============================] - 1s 955ms/step
Top similar review 1:
Review: i love this guitar. you can have a clean tone or a crunch tone which is perfect for what i play. it was my first bass and i got it at a local used gear shop. i have had it for a little over a year and it has been through a lot. it sounds great and it is really durable and can go through a lot. i have played this bass every where i have played
Cosine similarity of text: 0.9238791159456335
Image URL: ['https://images-na.ssl-images-amazon.com/images/I/51NB1g4ibdL._SY88.jpg']
Cosine similarity of image: 0.39137405
Composite similarity: 0.6576265837582629

Top similar review 2:
Review: this is great. i boug

### Ranking based on composite similarity scores

In [84]:
# Sort the list based on composite similarity scores
sorted_pairsText = sorted(composite_similarity_data,key=lambda x: x['composite'], reverse=True)

top_three_composite = sorted_pairsText[:3]
# Display the sorted list
for index, item in enumerate(top_three_composite, 1):
    print(f"Top similar review {index}:")
    print("Image URL:", item["image_url"])
    print("Review:", item["review_text"])
    print("Cosine similarity of text:", item["cosine_text"])
    print("Cosine similarity of image:", item["cosine_image"])
    print("Composite similarity:", item["composite"])
    print()

Top similar review 1:
Image URL: https://images-na.ssl-images-amazon.com/images/I/61Rv7lR1aTL._SY88.jpg
Review: A great product, which made a wonderful under-bed light in our application.

- Length is perfect for a kings size frame,  Start about 1 food from the headboard, and this will cover the permitter all the way around, with adjustment for your particular bed

- The dimming feature is great and a must if you will use it for a nightlight application.  In fact, the lights are so bright you have to keep them at or near the dimmest level for an under-bed nightlight application.

- Every LED lit up.  No dead diodes.

Why the deduction?

- The adhesive back could be stickier.  I used regular scotch tape to strength where it would not adhere.  I think it will stick best to non-textured and gloss surfaces.

- The lead wire going to an from the dimmer would ideally be about 3-6" longer on each end.

- Although I purchased the warm light, like most LEDs, the color temperature becomes cooler

In [132]:
# Specify the file path where you want to save the image features
pickle_file_path = 'composite_similarity_data.pkl'

# Serialize and save the image_features array to a file
with open(pickle_file_path, 'wb') as f:
    pickle.dump(composite_similarity_data, f)


In [133]:
with open('composite_similarity_data.pkl', 'rb') as f:
    composite_similarity_data = pickle.load(f)

### Observations

Based on the provided output, it is evident that the text retrieval technique yields higher composite similarity scores compared to image retrieval for the given input review and image URL.

Reasoning :

The higher composite similarity scores achieved by the text retrieval technique can be attributed to several factors:

1. Semantic Content: Textual reviews provide more detailed and nuanced descriptions of products or experiences compared to images. They contain rich semantic content, including features, functionalities, user experiences, and opinions. Therefore, similarity calculations based on text are likely to capture more relevant information, leading to higher similarity scores.

2. TF-IDF Representation: The TF-IDF representation used for text retrieval effectively captures the importance of each term in the reviews within the corpus. This allows for a more meaningful comparison between the input review and the database of reviews, leading to accurate similarity assessments.

3. Natural Language Processing (NLP) Techniques: Preprocessing steps such as tokenization, stop-word removal, stemming, and lemmatization enhance the quality of textual data and facilitate better matching of semantics between reviews. These NLP techniques contribute to the higher accuracy of text-based similarity calculations.



### Challenges

Challenges Faced:

While the text retrieval technique outperformed image retrieval in this scenario, there are challenges and limitations associated with both approaches:

1. Limited Information in Images: Images may lack detailed textual information, making it challenging to extract meaningful features directly. This limitation can lead to reduced accuracy in image-based similarity calculations, especially when images do not contain explicit textual content.

2. Subjectivity in Reviews: Textual reviews are subjective and may vary in length, style, and focus. Matching such diverse textual descriptions accurately requires robust natural language processing techniques and careful consideration of semantic similarities.

3. Dependency on Image Quality: Image retrieval accuracy heavily depends on the quality and relevance of the images available in the dataset. Low-quality or irrelevant images may lead to suboptimal similarity scores, impacting the overall performance of the system.



### Potential Improvements

Potential Improvements:

To address the challenges mentioned above and improve the retrieval process, several enhancements can be considered:

1. Integration of Multi-Modal Approaches: Combining both text and image modalities in a unified retrieval framework can leverage complementary information and enhance overall similarity assessments. Techniques such as multi-modal embeddings and fusion models can integrate textual and visual features effectively.

2. Fine-Tuning Pre-Trained Models: Fine-tuning pre-trained CNN models such as ResNet50 on domain-specific image datasets can improve feature extraction accuracy and relevance. Fine-tuning allows the model to adapt to specific image characteristics and optimize feature representations for similarity calculations.

3. Semantic Image Retrieval: Incorporating advanced image analysis techniques, such as object detection, scene recognition, and image captioning, can enrich image representations with semantic information. Semantic image retrieval approaches can bridge the gap between textual and visual content, enabling more accurate similarity assessments.

4. User Feedback Mechanisms: Implementing user feedback mechanisms to iteratively refine similarity calculations based on user preferences and relevance judgments can enhance the adaptability and personalization of the retrieval system. User feedback can guide the system in learning and improving its performance over time.